In [3]:
from google.colab import drive
drive.mount("/content/drive")
import nltk
nltk.download('punkt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


ParseError: ignored

In [4]:

import numpy as np
!pip install -U sentence-transformers
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate
import xgboost as xgb
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from nltk.util import ngrams
import re
from pathlib import Path
import string
from sklearn.model_selection import train_test_split
from functools import reduce
from math import log
import itertools
from tqdm import tqdm
import os
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
import json
from nltk.tokenize.casual import TweetTokenizer
import pickle
from gensim.models.word2vec import Word2Vec
import keras
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.metrics import precision_recall_fscore_support
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
import gensim.downloader as api
import gensim
from sklearn.model_selection import KFold



     |████████████████████████████████| 78 kB 5.2 MB/s 
     |████████████████████████████████| 3.1 MB 29.7 MB/s 
     |████████████████████████████████| 3.3 MB 51.4 MB/s 
     |████████████████████████████████| 1.2 MB 60.8 MB/s 
     |████████████████████████████████| 59 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 58.4 MB/s 
     |████████████████████████████████| 895 kB 77.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=aa9f62252d6166d25a3ffee00cbd401c5b9cf04f55d6af130fb151f278da30c2
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
dir="/content/drive/MyDrive/nlp_project/"
os.chdir(dir)
df=pd.read_csv("train.csv")
output=list(df["is_duplicate"])
output_np=np.array(output)

In [6]:

output_encode=to_categorical(output_np)
output_encode=np.array(output_encode)


In [7]:
open_file = open("embeddings_para_1", "rb")
embeddings1= pickle.load(open_file)
open_file.close()
open_file = open("embeddings_para_2", "rb")
embeddings2 = pickle.load(open_file)
open_file.close()

In [8]:
embeddings1=np.array(embeddings1)
embeddings2=np.array(embeddings2)

In [9]:
input=np.hstack((embeddings1,embeddings2))

In [10]:
print(input.shape)

(404348, 768)


In [11]:
accuracy=[]
loss=[]
predictions=[]
truth=[]
X_train, X_test, y_train, y_test = train_test_split(input, output_encode, test_size=0.25, random_state=42)
model = Sequential()
#model.add(Dense(1024, input_dim=768,activation='relu'))
model.add(Dense(512, input_dim=768,activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc'])
model.fit(X_train,y_train,epochs=6,validation_split=0.2,batch_size=64)
score=model.evaluate(X_test,y_test)
print(score[1])
accuracy.append(score[1])
loss.append(score[0])

y_pred = model.predict(X_test)
y_pred= np.argmax(y_pred,axis=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               393728    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 16)                1040      
                                                                 
 dense_5 (Dense)             (None, 2)                 34        
                                                                 
Total params: 567,282
Trainable params: 567,282
Non-trai

In [12]:
y_test=np.argmax(y_test,axis=1)
model.save("model_deploy")

INFO:tensorflow:Assets written to: model_deploy/assets


In [13]:
print("S_Bert+MLP")
print("Accuracy: ",accuracy_score(y_test, y_pred)*100)
print("Classification report:\n", classification_report(y_test, y_pred))
print("F1 Score:\n ",f1_score(y_test, y_pred)*100)

S_Bert+MLP
Accuracy:  82.38744843550604
Classification report:
               precision    recall  f1-score   support

           0       0.86      0.86      0.86     64078
           1       0.76      0.76      0.76     37009

    accuracy                           0.82    101087
   macro avg       0.81      0.81      0.81    101087
weighted avg       0.82      0.82      0.82    101087

F1 Score:
  75.98726801899008


In [26]:
i=X_train[2]
i=i.reshape(1,-1)
i.shape

(1, 768)

In [27]:
y_pred = model.predict(i)
y_pred= np.argmax(y_pred,axis=1)

In [28]:
y_pred

array([1])